In [ ]:
! pip uninstall lightgbm -y

Found existing installation: lightgbm 2.2.3
Uninstalling lightgbm-2.2.3:
  Successfully uninstalled lightgbm-2.2.3


In [ ]:
!pip install lightgbm --install-option=--gpu

/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:232: UserWarning: Disabling all use of wheels due to the use of --build-option / --global-option / --install-option.
  cmdoptions.check_install_build_global(options)
     |████████████████████████████████| 1.5 MB 4.4 MB/s 
Skipping wheel build for lightgbm, due to binaries being disabled for it.
    Running setup.py install for lightgbm ... done


In [ ]:
import lightgbm as lgb

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
cd '../gdrive/MyDrive/SSAC/3조'

/gdrive/.shortcut-targets-by-id/15_BxZVEQYCdGCGiQ5nexpWPc1cgHVe4w/3조


# CTR 예측 모델링

### 라이브러리 로드

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns

In [ ]:
import joblib
import os

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 학습 데이터 불러오기

In [ ]:
df = pd.read_csv('data/merged_data_notext.csv')
df.head()

,bid_price,imp_id,imp_time,label,user_id,view_time,adv_id,content_name,content_price,content_used,content_cat_1,content_cat_2,content_cat_3,content_emergency_count,content_comment_count,content_views,content_likes,content_b_pay,content_delivery_fee,content_place,content_status,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,viewer_gender,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count,content_img_url
0,50,99d1612dded122458e99,16:48:41.780,1,1728109,16:56:46.013,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...
1,50,971a612de1a50ebc01ce,17:00:45.805,0,1728109,NaN,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...
2,50,971a612de24b0ebe9859,17:03:49.363,0,1728109,NaN,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...
3,50,9919612de22422461f9b,17:02:48.690,0,1728109,NaN,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...
4,50,998c612de1e300aa29bc,17:01:51.038,0,1728109,NaN,12851816,#당일발송#아이폰SE2 64G/128G 대량판매 등급별초특가!,250000,1,전자제품,600700,600700001,1,4,25583,817,0,NaN,NaN,삭제됨,2131,5285,247,4525,542,739,496,0,0,11,1,30,5,0,0,0,0,https://media.bunjang.co.kr/product/137078818_...


In [ ]:
df.columns

Index(['bid_price', 'imp_id', 'imp_time', 'label', 'user_id', 'view_time',
       'adv_id', 'content_name', 'content_price', 'content_used',
       'content_cat_1', 'content_cat_2', 'content_cat_3',
       'content_emergency_count', 'content_comment_count', 'content_views',
       'content_likes', 'content_b_pay', 'content_delivery_fee',
       'content_place', 'content_status', 'adv_follower_count', 'adv_grade',
       'adv_item_count', 'adv_views', 'adv_review_count', 'adv_comment_count',
       'adv_pay_count', 'adv_parcel_post_count', 'adv_transfer_count',
       'adv_chat_count', 'viewer_gender', 'viewer_age',
       'viewer_following_count', 'viewer_pay_count',
       'viewer_parcel_post_count', 'viewer_transfer_count',
       'viewer_chat_count', 'content_img_url'],
      dtype='object')

In [ ]:
df['content_emergency_count'].describe()

count    735694.000000
mean          1.559269
std          11.270738
min           0.000000
25%           0.000000
50%           0.000000
75%           1.000000
max         273.000000
Name: content_emergency_count, dtype: float64

In [ ]:
df['viewer_age'].value_counts()

0      69636
25     20780
22     19652
26     19441
42     19353
       ...  
93         7
100        4
111        4
96         1
115        1
Name: viewer_age, Length: 93, dtype: int64

# EDA

## 전처리 함수 정의

In [ ]:
categorical = [
    'viewer_gender',  
    'content_used', 
    'content_cat_1', 
    'content_cat_2', 
    'content_cat_3']

In [ ]:
continuous = [
    'bid_price', 
    'content_price',
    'content_emergency_count',
    'content_comment_count',     
    'content_views',       
    'content_likes',    
    'adv_follower_count',
    'adv_grade',       
    'adv_item_count',         
    'adv_views',        
    'adv_review_count', 
    'adv_comment_count',
    'adv_pay_count',
    'adv_parcel_post_count', 
    'adv_transfer_count', 
    'adv_chat_count',
    'viewer_age',
    'viewer_following_count',
    'viewer_pay_count',
    'viewer_parcel_post_count', 
    'viewer_transfer_count',
    'viewer_chat_count']

In [ ]:
features = categorical + continuous

In [ ]:
def process_missing_values(df):
    for categorical_col in categorical:
        df[categorical_col] = df[categorical_col].astype(str)
        df[categorical_col] = df[categorical_col].fillna('0')
        df[categorical_col] = preprocessing.LabelEncoder().fit_transform(df[categorical_col])

    for continuous_col in continuous:
        df[continuous_col] = df[continuous_col].fillna(0)

    return df

## 전처리 진행

In [ ]:
df = process_missing_values(df)

### 평균 CTR 계산

In [ ]:
grouped_label = df.groupby('label').size()
average_ctr = float(grouped_label[1]/grouped_label.sum())
average_ctr

0.03200107653453746

## Train/Test 데이터 나누기 (8:2)

In [ ]:
train_test_df = df[['label'] + features]
train, test = train_test_split(train_test_df, test_size = 0.2, random_state=47)

X_train = train[features]
y_train = train['label']

X_test = test[features]
y_test = test['label']

In [ ]:
X_train.head()

,viewer_gender,content_used,content_cat_1,content_cat_2,content_cat_3,bid_price,content_price,content_emergency_count,content_comment_count,content_views,content_likes,adv_follower_count,adv_grade,adv_item_count,adv_views,adv_review_count,adv_comment_count,adv_pay_count,adv_parcel_post_count,adv_transfer_count,adv_chat_count,viewer_age,viewer_following_count,viewer_pay_count,viewer_parcel_post_count,viewer_transfer_count,viewer_chat_count
161683,1,1,17,64,123,50,375000,0,0,593,19,2131,5285,247,4525,542,739,496,0,0,11,41,0,0,0,0,0
665614,2,0,13,9,12,50,128000,0,0,78,3,949,637,728,2087,65,3,60,0,1,0,30,54,1,100,0,0
680267,2,0,13,10,15,50,47000,0,0,138,8,448,225,581,373,23,4,38,0,0,0,51,1,32,0,7,0
623185,0,0,11,49,91,60,280000,3,244,21926,683,7144,10000,85,357189,1030,948,282,0,437,4,63,8,0,0,0,0
329998,0,1,10,69,136,50,880000,0,0,406,2,357,546,403,1507,58,124,130,0,2,11,0,0,0,0,0,0


# LightGBM 학습

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from lightgbm import LGBMClassifier

LGBMClassifier(device='gpu', n_estimators=300)

## GridSearchCV

평가함수 정의

In [ ]:
from sklearn.metrics import roc_auc_score

def get_rig(train_y, test_y, pred):
    avg_ctr = average_ctr
    prior = log_loss(train_y, [avg_ctr]*len(train_y))

    classifier = log_loss(test_y, pred)

    rig = (prior - classifier) / prior
    return rig


params 설정

In [ ]:
#   'max_depth' : [10, 15, 20, 25, 30],
#   'num_leaves': [31, 63, 127], 
#   'learning_rate': [0.1, 0.05, 0.01, 0.005], 
p_list = []
for md in [5, 6, 7, 8, 9, 10, 11, 12]:
    for nl in [31, 50, 63, 100, 127]:
        for lr in [0.1, 0.05, 0.01]:
            p_list.append((lr, md, nl))
len(p_list)

120

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
from sklearn.model_selection import KFold
results = []
for lr, md, nl in p_list:
    folds = KFold(n_splits = 5, shuffle = True, random_state = 47)
    fold_aucs = []
    fold_rigs = []
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train)) :
        train_X, train_y = X_train.iloc[trn_idx], y_train.iloc[trn_idx] 
        valid_X, valid_y = X_train.iloc[val_idx], y_train.iloc[val_idx]
        
        params = { 'boosting_type': 'dart', 
                'metric': {'auc', 'binary_logloss'}, 
                'num_leaves': nl, 
                'max_depth': md,
                'learning_rate': lr, 
                'feature_fraction': 0.9, 
                'bagging_fraction': 0.8, 
                'bagging_freq': 5, 
                'verbose': 0} 

        # train_ds = lgb.Dataset(train_X, label=train_y)
        # valid_ds = lgb.Dataset(valid_X, label=valid_y) 

        model = LGBMClassifier(boosting_type='gbdt', num_leaves=nl, max_depth= md, learning_rate=lr, n_estimators=300, device = "gpu")
        print('start training')
        model.fit(train_X, train_y)
        print('predicting')
        pred = model.predict_proba(valid_X)[:,1]

        fold_aucs.append(roc_auc_score(valid_y, pred))
        fold_rigs.append(get_rig(train_y, valid_y, pred))
        print(f'auc : {roc_auc_score(valid_y, pred)}, rig: {get_rig(train_y, valid_y, pred)}')
    auc_score = sum(fold_aucs)/5
    rig_score = sum(fold_rigs)/5

    results.append({'num_leaves': nl, 'max_depth': md, 'learning_rate': lr, 'cv_auc': auc_score, 'cv_rig': rig_score})

start training
predicting
auc : 0.734765954191032, rig: 0.05958847866532905
start training
predicting
auc : 0.7413711240665464, rig: 0.11109309579422597
start training
predicting
auc : 0.7335006276261683, rig: 0.07703573321051597
start training
predicting
auc : 0.7421824265167251, rig: 0.08147782765904192
start training
predicting
auc : 0.7348159334681699, rig: 0.08592675040998228
start training
predicting
auc : 0.7290133128700145, rig: 0.05576642358169866
start training
predicting
auc : 0.7331756371286421, rig: 0.10511571445457149
start training
predicting
auc : 0.7274176908668215, rig: 0.07357595075075929
start training
predicting
auc : 0.7365231659191169, rig: 0.07688840011207454
start training
predicting
auc : 0.7285649487212652, rig: 0.081485635312028
start training
predicting
auc : 0.7084108083048732, rig: 0.038709196384849764
start training
predicting
auc : 0.7139201943887143, rig: 0.08949509678687206
start training
predicting
auc : 0.7070118585256473, rig: 0.057790954595904165


In [ ]:
results

[{'cv_auc': 0.7373272131737283,
  'cv_rig': 0.08302437714781905,
  'learning_rate': 0.1,
  'max_depth': 5,
  'num_leaves': 31},
 {'cv_auc': 0.730938951101172,
  'cv_rig': 0.0785664248422264,
  'learning_rate': 0.05,
  'max_depth': 5,
  'num_leaves': 31},
 {'cv_auc': 0.7120026574136913,
  'cv_rig': 0.06278014112219023,
  'learning_rate': 0.01,
  'max_depth': 5,
  'num_leaves': 31},
 {'cv_auc': 0.7372533513593114,
  'cv_rig': 0.08313468393201043,
  'learning_rate': 0.1,
  'max_depth': 5,
  'num_leaves': 50},
 {'cv_auc': 0.7304052844196371,
  'cv_rig': 0.07823909416164594,
  'learning_rate': 0.05,
  'max_depth': 5,
  'num_leaves': 50},
 {'cv_auc': 0.7119015702135496,
  'cv_rig': 0.06275187068599122,
  'learning_rate': 0.01,
  'max_depth': 5,
  'num_leaves': 50},
 {'cv_auc': 0.7374122021463025,
  'cv_rig': 0.08323355527715623,
  'learning_rate': 0.1,
  'max_depth': 5,
  'num_leaves': 63},
 {'cv_auc': 0.7304011870673479,
  'cv_rig': 0.07823735504819412,
  'learning_rate': 0.05,
  'max_depth

In [ ]:
grid_gbdt = pd.DataFrame(results)
grid_gbdt.to_csv('김동영/grid_dart.csv', index=False)

# 결과 분석

In [ ]:
df_dart = pd.read_csv('김동영/grid_dart.csv')
df_gbdt = pd.read_csv('김동영/grid_gbdt.csv')

In [ ]:
df_dart.sort_values(by='cv_auc', ascending=False)

,num_leaves,max_depth,learning_rate,cv_auc,cv_rig
57,127,8,0.10,0.749301,0.086181
114,100,12,0.10,0.749123,0.086821
118,127,12,0.05,0.748613,0.090554
99,100,11,0.10,0.748610,0.086497
87,127,10,0.10,0.748575,0.082788
...,...,...,...,...,...
2,31,5,0.01,0.712003,0.062780
14,127,5,0.01,0.711902,0.062752
5,50,5,0.01,0.711902,0.062752
8,63,5,0.01,0.711902,0.062752


In [ ]:
df_dart.sort_values(by='cv_rig', ascending=False)

,num_leaves,max_depth,learning_rate,cv_auc,cv_rig
118,127,12,0.05,0.748613,0.090554
115,100,12,0.05,0.747267,0.090360
88,127,10,0.05,0.748153,0.090307
103,127,11,0.05,0.748045,0.089950
73,127,9,0.05,0.747179,0.089624
...,...,...,...,...,...
2,31,5,0.01,0.712003,0.062780
14,127,5,0.01,0.711902,0.062752
8,63,5,0.01,0.711902,0.062752
11,100,5,0.01,0.711902,0.062752


In [ ]:
df_gbdt.sort_values(by='cv_auc', ascending=False)

,num_leaves,max_depth,learning_rate,cv_auc,cv_rig
72,127,9,0.10,0.724034,0.072089
57,127,8,0.10,0.723521,0.070972
87,127,10,0.10,0.723380,0.072171
102,127,11,0.10,0.722968,0.072209
117,127,12,0.10,0.722508,0.072171
...,...,...,...,...,...
14,127,5,0.01,0.693396,0.013652
5,50,5,0.01,0.693396,0.013652
11,100,5,0.01,0.693396,0.013652
8,63,5,0.01,0.693396,0.013652


In [ ]:
df_gbdt.sort_values(by='cv_rig', ascending=False)

,num_leaves,max_depth,learning_rate,cv_auc,cv_rig
102,127,11,0.10,0.722968,0.072209
117,127,12,0.10,0.722508,0.072171
87,127,10,0.10,0.723380,0.072171
72,127,9,0.10,0.724034,0.072089
57,127,8,0.10,0.723521,0.070972
...,...,...,...,...,...
8,63,5,0.01,0.693396,0.013652
14,127,5,0.01,0.693396,0.013652
5,50,5,0.01,0.693396,0.013652
11,100,5,0.01,0.693396,0.013652
